## 추론(예측) 기반 기법과 신경망

## word2vec : 워드투벡터
2013년 구글의 토마스미콜로프(Tomas Mikolov)의 팀이 개발<br>
<b>word2vec</b> 알고리즘은 <b>신경망 모델</b>을 사용 하여 큰 텍스트 코퍼스에서 단어 연관성을 학습. 학습이 끝나면 이러한 모델은 동의어 단어를 감지하거나 부분 문장에 대한 추가 단어를 제안 할 수 있다. word2vec는 <b>벡터</b> 라고하는 특정 숫자 목록을 사용하여 각각의 고유 한 단어를 나타낸다 . 벡터는 간단한 수학적 함수 ( 벡터 간의 코사인 유사성 ) 가 해당 벡터가 나타내는 단어 간의 의미 유사성 수준을 나타내 도록 신중하게 선택 된다.



## [1] 신경망에서의 단어 처리

In [1]:
import numpy as np

text = 'You say goodbye and I say hello.'
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# 여기서 'you'만  one-hot 인코딩으로 표현

c = np.array([[1,0,0,0,0,0,0]])
print('c:\n',c)
W = np.random.randn(7,3)
print('W:\n',W)

h = np.matmul(c,W)  # (1,7) * (7,3) = (1,3)
print('h:\n',h)      

c:
 [[1 0 0 0 0 0 0]]
W:
 [[-2.13456479 -2.28814684  0.66465447]
 [ 1.08618006 -1.47603176 -1.25909644]
 [-2.05522825 -0.74531851 -0.9637862 ]
 [-0.43319499 -0.45281204 -0.43535025]
 [ 0.12009533 -1.09221548 -0.50279585]
 [ 0.69144156 -0.06598601  0.57003814]
 [-0.97273341 -0.37095208 -0.87792992]]
h:
 [[-2.13456479 -2.28814684  0.66465447]]


In [2]:
class MatMul:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self,x):
        W, = self.params
        out = np.dot(x,W)
        self.x = x
        return out
        
    def backward(self,dout):
        W, =self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0] = dw
        return dx

In [3]:
text = 'You say goodbye and I say hello.'
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# 여기서 'you'만  one-hot 인코딩으로 표현

c = np.array([[1,0,0,0,0,0,0]])
print('c:\n',c)
W = np.random.randn(7,3)
print('W:\n',W)

layer = MatMul(W)
h = layer.forward(c)  # (1,7) * (7,3) = (1,3)
print('h:\n',h) 

c:
 [[1 0 0 0 0 0 0]]
W:
 [[ 1.19316067  1.39930595  0.91044293]
 [-0.2914987   0.59268941  0.5723308 ]
 [-0.28430797  1.74179194 -0.43336352]
 [ 2.07342554 -1.56300242 -0.35282029]
 [-0.13037114 -0.40083452 -1.68301595]
 [-0.86220691  0.94540248  0.46773446]
 [ 1.01901241 -0.28997704  0.01099561]]
h:
 [[1.19316067 1.39930595 0.91044293]]


## [2] 단순한 word2vec

### CBOW (Continuous Bag of Words) 모델

#### Word2Vec에는 CBOW(Continuous Bag of Words)와 Skip-Gram 두 가지 방식이 있다
- $ CBOW $ 는 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법 <br>
  타깃(target)은 중앙 단어 그 주변 단어들이 맥락(contexts)이다
- $ Skip-Gram $ 은 중간에 있는 단어로 주변 단어들을 예측하는 방법

#### BOW(Bag of Words) : 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법

BOW를 만드는 과정<br>
(1) 우선, 각 단어의 고유한 인덱스(Index)를 부여한다.<br>
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터(Vector)를 만든다.<br>

"정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."<br>
('정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9) <br>
BOW: [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]  ==> '가' 와 '물가상승률' 은 2회 발생

https://wikidocs.net/22650

In [4]:
# (CBOW 전체구조의 Preview)
# 샘플 맥락 데이터 : 2개의 주변 단어를 맥락으로 중간 단어('say')를 예측

text = 'You say goodbye and I say hello.'

# 2개의 주변 단어를 one-hot 벡터 생성
c0 = np.array([[1,0,0,0,0,0,0]])  # 'you'   , (1,7)
c1 = np.array([[0,0,1,0,0,0,0]])  # 'goodbye' (1,7)


# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)   
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)  # (1,7) * (7,3)  = (1,3)
h1 = in_layer0.forward(c1)  # (1,7) * (7,3)  = (1,3)
h = 0.5 * (h0 + h1)         # 입력층이 여러 개이면 전체를 평균
s = out_layer.forward(h)    # (1,3) * (3,7)  = (1,7) ,최종 출력
print(s,s.shape)            # (1, 7), 7개 단어의 스코어
np.argmax(s)

[[ 1.83786569  1.35859265 -0.84231118  1.10379911  0.12384351  0.56529255
   0.62452658]] (1, 7)


0

## [3] 학습 데이터 준비

### 맥락과 타깃을 생성

In [5]:
from mynlp import preprocess,create_co_matrix,cos_similarity,most_similar
import numpy as np
import matplotlib.pyplot as plt

you
say
goodbye
and
i
say
hello
.
[0 1 2 3 4 1 5 6]
[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
[0 1 2 3 4 1 5 6]
{'i': 0, 'like': 1, 'apple': 2, 'and': 3, 'you': 4, 'banana': 5, '.': 6}
[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
0.7071067691154799
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
[0 1 0 1 0 0 0]
[0 1 0 0 0 0 1]
0.49999999292893216

[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


In [6]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id,id_to_word = preprocess(text)
print(corpus)     # 8 개
print(id_to_word) # 7 개

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [7]:
# target : (6,)
target = corpus[1:-1]  # 타깃(중간단어) : [1 2 3 4 1 5], 첫번째와 마지막 단어 제외
print('target:',target)

# contexts : (2,6)
# print('corpus:',corpus)

contexts = []
for idx in(range(1,len(corpus) - 1)): # 1 to 6 ,6회, 중간단어마다 앞뒤 주변단어 조합 6가지 
    cs = []
    for t in range(-1,2) : # 3회 , (-2, 0 ,1)
        if t == 0: # 2번째는 skip
            continue
        cs.append(corpus[idx + t]) # corpus[1-1,1+1],[2-1,2+1],...
    contexts.append(cs)
print('contexts:',contexts)

target: [1 2 3 4 1 5]
contexts: [[0, 2], [1, 3], [2, 4], [3, 1], [4, 5], [1, 6]]


In [8]:
# contexts와 target을 구하는 함수
def create_contexts_target(corpus,window_size=1):
    target = corpus[window_size:-window_size]  

    contexts = []
    for idx in(range(window_size,len(corpus) -window_size)): 
        cs = []
        for t in range(-window_size,window_size+1) : 
            if t == 0: 
                continue
            cs.append(corpus[idx + t]) 
        contexts.append(cs)
    return np.array(contexts),np.array(target)  

In [9]:
contexts,target = create_contexts_target(corpus,window_size=1)
print(contexts)   # (6,2)

# 맥락(contexts) : 예측할 단어의 주변 단어
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# window_size=1 일 경우 : 주변 단어를 중간 단어에 앞,뒤로 1개만 사용
# [[0 2]   : 'you', 'goodbye'
#  [1 3]   : 'say', 'and'
#  [2 4]   : 'goodbye', 'i'
#  [3 1]   : 'and', 'say'
#  [4 5]   : 'i', 'hello'
#  [1 6]]  : 'say', '.'

# window_size=2 일 경우 : : 주변 단어를 중간 단어에 앞,뒤로 2개 사용
# [[0 1 3 4]
#  [1 2 4 1]
#  [2 3 1 5]
#  [3 4 5 6]]

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]


In [10]:
print(target)
# 타깃(target) : 예측할 단어, 중간단어, 6개
# ['say','goodbye','and','i','say','hello']
# [1 2 3 4 1 5]

[1 2 3 4 1 5]


### 맥락과 타깃을 원핫 표현으로 변환

In [11]:
# 원핫 변환 함수

# target [1 2 3 4 1 5]을 변환하는 경우를 주석으로 설명 
def convert_one_hot(corpus, vocab_size):  # [1 2 3 4 1 5], 7
    N = corpus.shape[0] # (6,) --> 6

    if corpus.ndim == 1: # target [1 2 3 4 1 5], 1차원인경우 ==> 2차원으로 출력
        one_hot = np.zeros((N, vocab_size), dtype=np.int32) # 0으로 초기화된 (6,7) 2차원 배열 생성 
        for idx, word_id in enumerate(corpus): # 6회 반복
            one_hot[idx, word_id] = 1  # one_hot[0,1] = 1, [1,2]=1, [2,3] = 1,...,  [3,4],[4,1],[5,5] = 1...

    elif corpus.ndim == 2: # contexts 2차원 인경우 ==> 3차원으로 출력
        C = corpus.shape[1] # (6,2) --> 2
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32) # 0으로 초기화된 (6,2,7) 3차원 배열 생성 
        for idx_0, word_ids in enumerate(corpus): # 6회
            for idx_1, word_id in enumerate(word_ids): #  2회
                one_hot[idx_0, idx_1, word_id] = 1  

    return one_hot

In [12]:
vocab_size = len(word_to_id)
print(vocab_size)  # 7
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

7


In [13]:
print(contexts)
print(contexts.shape) # (6, 2, 7)

[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]
(6, 2, 7)


In [14]:
print(target)
print(target.shape)  # (6, 7)

[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]
(6, 7)


## [4] CBOW 신경망 모델 구현

In [15]:
from nn_layers import  MatMul, SoftmaxWithLoss, Adam

[[0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]] (1, 8)
----------------------------------------------------------------------
[[0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]
 [0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]
 [0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]
 [0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]
 [0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]
 [0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]
 [0.70702157 0.70317994 0.04310258 0.77840388 0.22215436 0.83712777
  0.79583412 0.62756067]] (7, 8)
[[0.38921343 0.96347074 0.8086814  0.70764847 0.1486479  0.83955879
  0.35708212 0.17879931]
 [0.01859391 0.49326765 0.83734449 0.20989691 0.78045599 0.76112536
  0.5805

In [16]:
# #  다중 분류 모델 : Softmax 사용
# class SimpleCBOW :
#     def __init__(self,vocab_size,hidden_size):  # 어휘수 : 7개, 은닉층의 뉴런 : 5
#         V,H = vocab_size,hidden_size
        
#         # 가중치 초기화
#         W_in = 0.01*np.random.randn(V,H).astype('f')   # (7,5)
#         W_out = 0.01*np.random.randn(H,V).astype('f')  # (5,7)
        
#         # 계층 생성
#         self.in_layer0 = MatMul(W_in)
#         self.in_layer1 = MatMul(W_in)
#         self.out_layer = MatMul(W_out)
#         self.loss_layer = SoftmaxWithLoss()
        
#         # 모든 가중치와 기울기를 리스트에 모은다.
#         layers = [self.in_layer0,self.in_layer1,self.out_layer,self.loss_layer]
#         self.params,self.grads = [],[]
#         for layer in layers: # 4회
#             self.params += layer.params
#             self.grads += layer.grads
            
#         # 인스턴스 변수에 단어의 분산 표현을 저장한다.
#         self.word_vec = W_in
        
#     def predict(self,contexts):  # contexts : (6,2,7)
        
#         # (6,7) * (7,5) = (6,5)
#         h0 = self.in_layer0.forward(contexts[:,0])  # (6,7)으로 입력, 맥락의 첫번째 단어 
#         h1 = self.in_layer0.forward(contexts[:,1])  # (6,7)으로 입력, 맥락의 두번째 단어
        
#         h = (h0 + h1) * 0.5  # 평균
        
#         # (6,5) * (5,7) = (6,7)
#         score = self.out_layer.forward(h)
#         return self.loss_layer.softmax(score) # softmax()함수로 확률값으로 출력
        
#     def forward(self,contexts,target):  # contexts : (6,2,7) , target : (6,7)
        
#         # (6,7) * (7,5) = (6,5)
#         h0 = self.in_layer0.forward(contexts[:,0])  # (6,7)으로 입력, 맥락의 첫번째 단어 
#         h1 = self.in_layer0.forward(contexts[:,1])  # (6,7)으로 입력, 맥락의 두번째 단어
        
#         h = (h0 + h1) * 0.5  # 평균
        
#         # (6,5) * (5,7) = (6,7)
#         score = self.out_layer.forward(h)
#         loss = self.loss_layer.forward(score,target)
#         return loss
    
#     def backward(self,dout=1):
#         ds = self.loss_layer.backward(dout)
#         da = self.out_layer.backward(ds)
#         da *= 0.5
#         self.in_layer0.backward(da)
#         self.in_layer1.backward(da)
#         return None        

In [17]:
# 다중 분류 모델 : Softmax 사용
class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size): # 어휘 수 : 7개, 은닉 층의 뉴런수 : 5
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in
        
    def predict(self,contexts):  # contexts : (6,2,7)
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        return self.loss_layer.softmax(score) # softmax로 변화
        
    def forward(self, contexts, target):  # contexts : (6,2,7) , target : (6,7)
         # (6,7) * (7,5) = (6,5)
        h0 = self.in_layer0.forward(contexts[:, 0]) # (6,7)으로 입력, 맥락의 첫번째 단어 
        h1 = self.in_layer1.forward(contexts[:, 1]) # (6,7)으로 입력, 맥락의 두번째 단어
        h = (h0 + h1) * 0.5   # 평균
        
        # (6,5) * (5,7) = (6,7)
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss
    
    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None

### Trainer 클래스


In [18]:
import time

def remove_duplicate(params, grads):
    '''
    매개변수의 중복 제거 함수
    매개변수 배열 중 중복되는 가중치를 하나로 모아
    그 가중치에 대응하는 기울기를 더한다.
    '''
    params, grads = params[:], grads[:]  # copy list

    while True:
        find_flg = False
        L = len(params)

        for i in range(0, L - 1):
            for j in range(i + 1, L):
                # 가중치 공유 시
                if params[i] is params[j]:
                    grads[i] += grads[j]  # 경사를 더함
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)
                # 가중치를 전치행렬로 공유하는 경우(weight tying)
                elif params[i].ndim == 2 and params[j].ndim == 2 and \
                     params[i].T.shape == params[j].shape and np.all(params[i].T == params[j]):
                    grads[i] += grads[j].T
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)

                if find_flg: break
            if find_flg: break

        if not find_flg: break

    return params, grads


class Trainer:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        self.loss_list = []
        self.eval_interval = None
        self.current_epoch = 0

    def fit(self, x, t, max_epoch=10, batch_size=32, max_grad=None, eval_interval=20):
        data_size = len(x)
        max_iters = data_size // batch_size
        self.eval_interval = eval_interval
        model, optimizer = self.model, self.optimizer
        total_loss = 0
        loss_count = 0

        start_time = time.time()
        for epoch in range(max_epoch):
            # 뒤섞기
            idx = np.random.permutation(np.arange(data_size))
            x = x[idx]
            t = t[idx]

            for iters in range(max_iters):
                batch_x = x[iters*batch_size:(iters+1)*batch_size]
                batch_t = t[iters*batch_size:(iters+1)*batch_size]

                # 기울기 구해 매개변수 갱신
                loss = model.forward(batch_x, batch_t)
                model.backward()
                
                params, grads = remove_duplicate(model.params, model.grads)  # 공유된 가중치를 하나로 모음
                if max_grad is not None:
                    clip_grads(grads, max_grad)
                optimizer.update(params, grads)
                total_loss += loss
                loss_count += 1

                # 평가
                if (eval_interval is not None) and (iters % eval_interval) == 0:
                    avg_loss = total_loss / loss_count
                    elapsed_time = time.time() - start_time
                    print('| 에폭 %d |  반복 %d / %d | 시간 %d[s] | 손실 %.2f'
                          % (self.current_epoch + 1, iters + 1, max_iters, elapsed_time, avg_loss))
                    self.loss_list.append(float(avg_loss))
                    total_loss, loss_count = 0, 0

            self.current_epoch += 1

    def plot(self, ylim=None):
        x = np.arange(len(self.loss_list))
        if ylim is not None:
            plt.ylim(*ylim)
        plt.plot(x, self.loss_list, label='train')
        plt.xlabel('반복 (x' + str(self.eval_interval) + ')')
        plt.ylabel('손실')
        plt.show()      
    

In [19]:
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)

| 에폭 1 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 2 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 3 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 4 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 5 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 6 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 7 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 8 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 9 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 10 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 11 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 12 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 13 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 14 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 15 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 16 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 17 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 18 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 19 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 20 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 21 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 22 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 23 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 24 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 25 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 26 |

| 에폭 233 |  반복 1 / 2 | 시간 0[s] | 손실 1.10
| 에폭 234 |  반복 1 / 2 | 시간 0[s] | 손실 1.05
| 에폭 235 |  반복 1 / 2 | 시간 0[s] | 손실 1.21
| 에폭 236 |  반복 1 / 2 | 시간 0[s] | 손실 1.02
| 에폭 237 |  반복 1 / 2 | 시간 0[s] | 손실 1.12
| 에폭 238 |  반복 1 / 2 | 시간 0[s] | 손실 1.17
| 에폭 239 |  반복 1 / 2 | 시간 0[s] | 손실 1.03
| 에폭 240 |  반복 1 / 2 | 시간 0[s] | 손실 1.03
| 에폭 241 |  반복 1 / 2 | 시간 0[s] | 손실 1.18
| 에폭 242 |  반복 1 / 2 | 시간 0[s] | 손실 1.09
| 에폭 243 |  반복 1 / 2 | 시간 0[s] | 손실 1.00
| 에폭 244 |  반복 1 / 2 | 시간 0[s] | 손실 1.08
| 에폭 245 |  반복 1 / 2 | 시간 0[s] | 손실 1.14
| 에폭 246 |  반복 1 / 2 | 시간 0[s] | 손실 1.02
| 에폭 247 |  반복 1 / 2 | 시간 0[s] | 손실 1.22
| 에폭 248 |  반복 1 / 2 | 시간 0[s] | 손실 0.99
| 에폭 249 |  반복 1 / 2 | 시간 0[s] | 손실 1.00
| 에폭 250 |  반복 1 / 2 | 시간 0[s] | 손실 1.05
| 에폭 251 |  반복 1 / 2 | 시간 0[s] | 손실 1.16
| 에폭 252 |  반복 1 / 2 | 시간 0[s] | 손실 1.04
| 에폭 253 |  반복 1 / 2 | 시간 0[s] | 손실 1.05
| 에폭 254 |  반복 1 / 2 | 시간 0[s] | 손실 1.13
| 에폭 255 |  반복 1 / 2 | 시간 0[s] | 손실 0.97
| 에폭 256 |  반복 1 / 2 | 시간 0[s] | 손실 1.04
| 에폭 257 |  반복 1

| 에폭 473 |  반복 1 / 2 | 시간 0[s] | 손실 0.62
| 에폭 474 |  반복 1 / 2 | 시간 0[s] | 손실 0.61
| 에폭 475 |  반복 1 / 2 | 시간 0[s] | 손실 0.69
| 에폭 476 |  반복 1 / 2 | 시간 0[s] | 손실 0.59
| 에폭 477 |  반복 1 / 2 | 시간 0[s] | 손실 0.53
| 에폭 478 |  반복 1 / 2 | 시간 0[s] | 손실 0.60
| 에폭 479 |  반복 1 / 2 | 시간 0[s] | 손실 0.76
| 에폭 480 |  반복 1 / 2 | 시간 0[s] | 손실 0.51
| 에폭 481 |  반복 1 / 2 | 시간 0[s] | 손실 0.69
| 에폭 482 |  반복 1 / 2 | 시간 0[s] | 손실 0.43
| 에폭 483 |  반복 1 / 2 | 시간 0[s] | 손실 0.68
| 에폭 484 |  반복 1 / 2 | 시간 0[s] | 손실 0.42
| 에폭 485 |  반복 1 / 2 | 시간 0[s] | 손실 0.67
| 에폭 486 |  반복 1 / 2 | 시간 0[s] | 손실 0.68
| 에폭 487 |  반복 1 / 2 | 시간 0[s] | 손실 0.50
| 에폭 488 |  반복 1 / 2 | 시간 0[s] | 손실 0.60
| 에폭 489 |  반복 1 / 2 | 시간 0[s] | 손실 0.66
| 에폭 490 |  반복 1 / 2 | 시간 0[s] | 손실 0.50
| 에폭 491 |  반복 1 / 2 | 시간 0[s] | 손실 0.67
| 에폭 492 |  반복 1 / 2 | 시간 0[s] | 손실 0.60
| 에폭 493 |  반복 1 / 2 | 시간 0[s] | 손실 0.49
| 에폭 494 |  반복 1 / 2 | 시간 0[s] | 손실 0.59
| 에폭 495 |  반복 1 / 2 | 시간 0[s] | 손실 0.65
| 에폭 496 |  반복 1 / 2 | 시간 0[s] | 손실 0.59
| 에폭 497 |  반복 1

| 에폭 709 |  반복 1 / 2 | 시간 0[s] | 손실 0.40
| 에폭 710 |  반복 1 / 2 | 시간 0[s] | 손실 0.51
| 에폭 711 |  반복 1 / 2 | 시간 0[s] | 손실 0.34
| 에폭 712 |  반복 1 / 2 | 시간 0[s] | 손실 0.37
| 에폭 713 |  반복 1 / 2 | 시간 0[s] | 손실 0.54
| 에폭 714 |  반복 1 / 2 | 시간 0[s] | 손실 0.20
| 에폭 715 |  반복 1 / 2 | 시간 0[s] | 손실 0.57
| 에폭 716 |  반복 1 / 2 | 시간 0[s] | 손실 0.34
| 에폭 717 |  반복 1 / 2 | 시간 0[s] | 손실 0.40
| 에폭 718 |  반복 1 / 2 | 시간 0[s] | 손실 0.36
| 에폭 719 |  반복 1 / 2 | 시간 0[s] | 손실 0.39
| 에폭 720 |  반복 1 / 2 | 시간 0[s] | 손실 0.40
| 에폭 721 |  반복 1 / 2 | 시간 0[s] | 손실 0.44
| 에폭 722 |  반복 1 / 2 | 시간 0[s] | 손실 0.36
| 에폭 723 |  반복 1 / 2 | 시간 0[s] | 손실 0.33
| 에폭 724 |  반복 1 / 2 | 시간 0[s] | 손실 0.50
| 에폭 725 |  반복 1 / 2 | 시간 0[s] | 손실 0.35
| 에폭 726 |  반복 1 / 2 | 시간 0[s] | 손실 0.40
| 에폭 727 |  반복 1 / 2 | 시간 0[s] | 손실 0.49
| 에폭 728 |  반복 1 / 2 | 시간 0[s] | 손실 0.30
| 에폭 729 |  반복 1 / 2 | 시간 0[s] | 손실 0.43
| 에폭 730 |  반복 1 / 2 | 시간 0[s] | 손실 0.39
| 에폭 731 |  반복 1 / 2 | 시간 0[s] | 손실 0.30
| 에폭 732 |  반복 1 / 2 | 시간 0[s] | 손실 0.38
| 에폭 733 |  반복 1

| 에폭 959 |  반복 1 / 2 | 시간 0[s] | 손실 0.30
| 에폭 960 |  반복 1 / 2 | 시간 0[s] | 손실 0.31
| 에폭 961 |  반복 1 / 2 | 시간 0[s] | 손실 0.43
| 에폭 962 |  반복 1 / 2 | 시간 0[s] | 손실 0.31
| 에폭 963 |  반복 1 / 2 | 시간 0[s] | 손실 0.11
| 에폭 964 |  반복 1 / 2 | 시간 0[s] | 손실 0.42
| 에폭 965 |  반복 1 / 2 | 시간 0[s] | 손실 0.18
| 에폭 966 |  반복 1 / 2 | 시간 0[s] | 손실 0.55
| 에폭 967 |  반복 1 / 2 | 시간 0[s] | 손실 0.20
| 에폭 968 |  반복 1 / 2 | 시간 0[s] | 손실 0.30
| 에폭 969 |  반복 1 / 2 | 시간 0[s] | 손실 0.31
| 에폭 970 |  반복 1 / 2 | 시간 0[s] | 손실 0.33
| 에폭 971 |  반복 1 / 2 | 시간 0[s] | 손실 0.30
| 에폭 972 |  반복 1 / 2 | 시간 0[s] | 손실 0.21
| 에폭 973 |  반복 1 / 2 | 시간 0[s] | 손실 0.41
| 에폭 974 |  반복 1 / 2 | 시간 0[s] | 손실 0.21
| 에폭 975 |  반복 1 / 2 | 시간 0[s] | 손실 0.42
| 에폭 976 |  반복 1 / 2 | 시간 0[s] | 손실 0.31
| 에폭 977 |  반복 1 / 2 | 시간 0[s] | 손실 0.29
| 에폭 978 |  반복 1 / 2 | 시간 0[s] | 손실 0.31
| 에폭 979 |  반복 1 / 2 | 시간 0[s] | 손실 0.43
| 에폭 980 |  반복 1 / 2 | 시간 0[s] | 손실 0.20
| 에폭 981 |  반복 1 / 2 | 시간 0[s] | 손실 0.30
| 에폭 982 |  반복 1 / 2 | 시간 0[s] | 손실 0.31
| 에폭 983 |  반복 1